# Hadoop Streaming assignment 2: Stop Words

The purpose of this task is to improve the previous "Word rating" program. You have to calculate how many stop words are there in the input dataset. Stop words list is in `/datasets/stop_words_en.txt` file. 

Use Hadoop counters to compute the number of stop words and total words in the dataset. The result is the percentage of stop words in the entire dataset (without percent symbol).

There are several points for this task:

1) As an output, you have to get the percentage of stop words in the entire dataset without percent symbol (correct answer on sample dataset is `41.603`).

2) As you can see in the Hadoop Streaming userguide "you will need to use `-files` option to tell the framework to pack your executable files as a part of a job submission."

3) Do not forget to redirect junk output to `/dev/null`.

4) You may modify mappers/reducers from "Word rating" task and parse its output to get the answer on "Stop Words" task.

5) You may use mapper/reducer to get `"Stop Words"` and `"Total Words"` amounts and redirect them to sys.stderr. After that you may redirect the output of MapReduce to the parsed function. In this function you may find rows correspond to these amounts and compute the percentage.

Here you can find the draft for the main steps of the task. You can use other methods to get the solution.

## Step 1. Create the mapper.

<b>Hint:</b> Create the mapper, which calculates Total word and Stop word amounts. You may redirect this information to sys.stderr. This will make it possible to parse these data on the next steps.

Example of the redirections:

`print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % count`

Remember about the Distributed cache. If we add option `-files mapper.py,reducer.py,/datasets/stop_words_en.txt`, then `mapper.py, reducer.py` and `stop_words_en.txt` file will be in the same directory on the datanodes. Hence, it is necessary to use a relative path `stop_words_en.txt` from the mapper to access this txt file.

In [ ]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
#        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

In [ ]:
%%writefile mapper_cnt.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

path = 'stop_words_en.txt'

def read_stop_words(file_path):
    return set(word.strip() for word in open(file_path))

stop_words = read_stop_words(path)

for line in sys.stdin:
    try:
        text, count_of_word = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    #print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
    count_of_word = int(count_of_word)
    if text in stop_words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % count_of_word
    print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % count_of_word
    print "%d\t%s" % (count_of_word, text.lower())

## Step 2. Create the reducer.

Create the reducer, which will accumulate the information after the mapper step. You may implement the combiner if you want. It can be useful from optimizing and speed up your computations (see the lectures from the Week 2 for more details).

In [ ]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

## Step 3. Create the parsed function.

<b>Hint:</b> Create the function, which will parse MapReduce sys.stderr for Total word and Stop word amounts.

The `./counter_process.py` script should do the following:

- parse hadoop logs from Stderr,

- retrieve values of 2 user-defined counters,

- compute percentage and output it into the stdout.

In [ ]:
%%writefile counter_process.py

#! /usr/bin/env python

import sys

stop_words = 0
total_words = 0

# Your functions may be here.

for line in sys.stdin:
    try:
        param, num = line.strip().split('=', 1)
        num = int(num)
    except ValueError as e:
        continue
    if param == sys.argv[1]:
        stop_words = num
    elif param == sys.argv[2]:
        total_words = num

percentage = float(stop_words) / float(total_words) * 100.0
print >> sys.stdout, "%f" % percentage

#if __name__ == '__main__':
    #print(percentage)

In [ ]:
%%writefile sorter.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

path = 'stop_words_en.txt'

def read_stop_words(file_path):
    return set(word.strip() for word in open(file_path))

stop_words = read_stop_words(path)

for line in sys.stdin:
    try:
        text, count_of_word = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    count_of_word = int(count_of_word)
    if text in stop_words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % count_of_word
    print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % count_of_word
    print "%d\t%s" % (count_of_word, text.lower())

In [ ]:
%%writefile reducer_step2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        count_of_word, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    #print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
    count_of_word = int(count_of_word)
    print "%s\t%d" % (text.lower(), count_of_word)

## Step 4. Bash commands

<b> Hints: </b> 

1) If you want to redirect standard output to txt file you may use the following argument in yarn jar:

```
yarn ... \
  ... \
  -output ${OUT_DIR} > /dev/null 2> $LOGS
```

2) For printing the percentage of stop words in the entire dataset you may parse the MapReduce output. Parsed script may be written in Python code. 

To get the result you may use the UNIX pipe operator `|`. The output of the first command acts as an input to the second command (see lecture file-content-exploration-2 for more details).

With this operator you may use command `cat` to redirect the output of MapReduce to ./counter_process.py with arguments, which correspond to the `"Stop words"` and `"Total words"` counters. Example is the following:

`cat $LOGS | python ./counter_process.py "Stop words" "Total words"`

Now something about Hadoop counters naming. 
 - Built-in Hadoop counters usually have UPPER_CASE names. To make the grading system possible to distinguish your custom counters and system ones please use the following pattern for their naming: `[Aa]aaa...` (all except the first letters should be in lowercase);
 - Another points is how Hadoop sorts the counters. It sorts them lexicographically. Grading system reads your first counter as Stop words counter and the second as Total words. Please name you counters in such way that Hadoop set the Stop words counter before the Total words. 
 
E.g. "Stop words" and "Total words" names are Ok because they correspond both requirements.

3) In Python code sys.argv is a list, which contains the command-line arguments passed to the script. The name of the script is in `sys.argv[0]`. Other arguments begin from `sys.argv[1]`.

Hence, if you have two arguments, which you send from the Bash to your python script, you may use arguments in your script with the following command:

`function(sys.argv[1], sys.argv[2])`

4) Do not forget about printing your MapReduce output in the last cell. You may use the next command:

`cat $LOGS >&2`

In [ ]:
%%bash

INPUT_DIR="coursera_mr_task2_step1"$(date +"%s%6N")
MID_DIR="coursera_mr_task2_step2"$(date +"%s%6N")
OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
NUM_REDUCERS=8
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${INPUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${MID_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=$NUM_REDUCERS \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${INPUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=1 \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D map.output.key.field.separator \
    -D mapreduce.partition.keycomparator.options=-k1nr \
    -files sorter.py,/datasets/stop_words_en.txt \
    -mapper "python sorter.py" \
    -input ${INPUT_DIR} \
    -output ${MID_DIR} > /dev/null 2> $LOGS

#yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
#    -D mapred.jab.name="Streaming wordCount" \
#    -D mapreduce.job.reduces=1 \
#    -files mapper_cnt.py,/datasets/stop_words_en.txt \
#    -mapper "python mapper_cnt.py" \
#    -input ${MID_DIR} \
#    -output ${OUT_DIR} > /dev/null 2> $LOGS
    
cat $LOGS | python ./counter_process.py "Stop words" "Total words"
cat $LOGS >&2

#hdfs dfs -rm -r -skipTrash ${INPUT_DIR} > /dev/null
#hdfs dfs -rm -r -skipTrash ${MID_DIR} > /dev/null
#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
